In [1]:
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.ensemble import RandomForestRegressor as RFR
RFR=RFR(max_depth=1)
from sklearn.linear_model import LinearRegression as LR
LR=LR()
from sklearn import metrics
import matplotlib.pyplot as plt
l=11 # Comprimento do teste em anos

i=pd.read_csv('~/git/BD/temp/imp_fert_quanti.csv').sort_values(by=['CO_ANO','CO_MES'])
i=i[i.SG_UF_NCM=='BA']
i['date']=pd.to_datetime(i.CO_ANO.astype(str)+'/'+i.CO_MES.astype(str)+'/1')
i=i.groupby(by=['date','CO_MES'])['KG_LIQUIDO'].sum().reset_index().sort_values('date')
i=i.sort_values('date')

In [2]:
cat_pred=i[i.date==i.date.max()].CO_MES.values[0]
cat_pred

np.int64(5)

In [3]:
i=i[i.CO_MES==cat_pred]

In [4]:
def predict(i_0):
    i=i_0.copy()
    
    pred=pd.DataFrame()
    
    while (i.date.max()-i.date.min()).days/365.25>=l:

        
        # Limita comprimento em anos
        j=i[i.date>i.date.max()-pd.DateOffset(years=l)]

        # Tipifica a data para processamento ML
        j.date=j.date.astype(int)

        # Limita categorias do traino as categorias presentes no teste
        cat_pred=j[j.date==j.date.max()].CO_MES.values[0]
        j=j[j.CO_MES==cat_pred]
        
        # Codifica categorias
        #j.CO_MES=j.CO_MES.astype('category')
        #j=pd.get_dummies(j,prefix_sep='~')
    
        # Identifica as variáveis
        train=j[j.date<j.date.max()]
        test=j[j.date==j.date.max()]
        X_train=train.drop(columns='KG_LIQUIDO')
        y_train=train['KG_LIQUIDO']
        X_test=test.drop(columns='KG_LIQUIDO')
        y_test=test['KG_LIQUIDO']

        # Prediz
        RFR.fit(X_train,y_train)
        q=X_test.assign(kg_pred=RFR.predict(X_test)).assign(KG_LIQUIDO=y_test)
        pred=pd.concat([q,pred])
        i=i[i.date<i.date.max()]
    
    return pred

In [5]:
pred=pd.DataFrame()
for re in range(30):
    q=predict(i)
    r=round(metrics.r2_score(q.KG_LIQUIDO,q.kg_pred),2)
    pred=pd.concat([pd.DataFrame({'score':[r]}),pred])

In [6]:
round(pred.score.mean(),2), round(pred.score.std(),2)

(np.float64(0.21), np.float64(0.02))